# ASR DeepSpeech2 Demo Notebook

Этот notebook демонстрирует запуск обучения и inference модели DeepSpeech2 для задачи ASR.

**Репозиторий:** https://github.com/Vladislav747/hw2-dl-audio


## 1. Клонирование репозитория и установка зависимостей

### Для приватных репозиториев:

Если репозиторий приватный, вам нужно:
1. Создать GitHub Personal Access Token: https://github.com/settings/tokens
2. Выдать права `repo` (для доступа к приватным репозиториям)
3. Использовать токен в команде клонирования (см. код ниже)


In [ ]:
# Клонируем репозиторий
import os
import getpass

REPO_URL = "https://github.com/Vladislav747/hw2-dl-audio.git"

# Для приватного репозитория раскомментируйте строки ниже:
# token = getpass.getpass('Введите GitHub Personal Access Token: ')
# REPO_URL = f"https://{token}@github.com/Vladislav747/hw2-dl-audio.git"

if not os.path.exists('hw2-dl-audio'):
    !git clone {REPO_URL}
    %cd hw2-dl-audio
else:
    %cd hw2-dl-audio


In [ ]:

!pip install -q torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0
!pip install -q soundfile numpy==1.26.4 scipy torch_audiomentations tqdm matplotlib pandas
!pip install -q hydra-core wget datasets editdistance pyctcdecode
!pip install -q comet_ml wandb


## 2. Быстрый тест на onebatchtest датасете


In [ ]:
# Запуск обучения на маленьком датасете для проверки
!python3 train.py -cn=deepspeech2 \
  datasets=onebatchtest \
  trainer.n_epochs=1 \
  trainer.override=True \
  writer=cometml


## 3. Обучение на полном датасете (train-clean-100)

**ВНИМАНИЕ:** Это займет много времени и места на диске! Датасет train-clean-100 весит ~6GB


In [ ]:
# Раскомментируйте для запуска полного обучения
# !python3 train.py -cn=deepspeech2 \
#   datasets=example \
#   trainer.n_epochs=50 \
#   trainer.override=True \
#   writer=cometml


## 4. Запуск Inference


In [ ]:
# Запуск inference на обученной модели
# Убедитесь, что у вас есть сохраненная модель в saved/model_best.pth

!python3 inference.py \
  -cn=inference \
  datasets=onebatchtest \
  inferencer.from_pretrained="saved/model_best.pth" \
  inferencer.save_path="predictions_demo"


## 5. Просмотр результатов


In [ ]:
# Просмотр сохраненных предсказаний
import torch
from pathlib import Path

predictions_dir = Path("data/saved/predictions_demo")
if predictions_dir.exists():
    for pred_file in sorted(predictions_dir.glob("**/output_*.pth")):
        pred = torch.load(pred_file)
        print(f"\n{pred_file.name}:")
        print(f"  Target: {pred.get('target', 'N/A')}")
        print(f"  Prediction (argmax): {pred.get('prediction_argmax', 'N/A')}")
        print(f"  Prediction (beam): {pred.get('prediction_beam', 'N/A')}")
else:
    print("Predictions directory not found. Run inference first.")


## 6. Настройка CometML (опционально)

Для логирования экспериментов в CometML нужно установить API ключ. Получить можно на https://www.comet.com/settings


In [ ]:
# Установите ваш CometML API ключ
# import os
# os.environ['COMET_API_KEY'] = 'your_api_key_here'
